### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [1]:
# !pip install langchain

In [1]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")

In [4]:
#!pip install langchain_groq

In [5]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001341F9A91C0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001341F9A9D30>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
# !pip install langchain_core

In [8]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [11]:
result

AIMessage(content='Here are a few ways to translate "Hello, how are you?" to French:\n\n* **Bonjour, comment allez-vous ?** (Formal, polite)\n* **Salut, ça va ?** (Informal, friendly)\n* **Coucou, comment vas-tu ?** (Very informal, used with friends/family)\n\n\nChoose the option that best fits the context and your relationship with the person you\'re speaking to.\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 21, 'total_tokens': 113, 'completion_time': 0.167272727, 'prompt_time': 0.00017732, 'queue_time': 0.020756529, 'total_time': 0.167450047}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2a64c73b-ceeb-46bd-8fae-6ffb5a16fec6-0', usage_metadata={'input_tokens': 21, 'output_tokens': 92, 'total_tokens': 113})

In [12]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'Here are a few ways to translate "Hello, how are you?" to French:\n\n* **Bonjour, comment allez-vous ?** (Formal, polite)\n* **Salut, ça va ?** (Informal, friendly)\n* **Coucou, comment vas-tu ?** (Very informal, used with friends/family)\n\n\nChoose the option that best fits the context and your relationship with the person you\'re speaking to.\n'

In [13]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'Here\'s how to translate "Hello, How are you?" to French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?**  (This is the most polite and formal way to say it.)\n\n**Informal:**\n\n* **Salut, ça va ?** (This is a casual and friendly way to greet someone.)\n\n\nLet me know if you have any other phrases you\'d like me to translate! \n'

In [14]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [17]:
result=prompt.invoke({"language":"French","text":"Hello"})

In [18]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [22]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"Hindi","text":"Hello"})

'नमस्ते (Namaste) \n'

In [23]:
#!pip install streamlit